In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # 每一行独立变量都会打印  而不仅仅是最后一行

# 位运算相关

```text
1）负数怎么用二进制表达
   打印二进制；直接定义二进制、十六进制的变量
   相反数（整数最小值的特殊性(取绝对值还是自己)）
2）常见的位运算（|、&、^、~、<<、>>、>>>）
3）为什么这么设计二进制（为了加法的逻辑是一套逻辑，没有条件转移）-> 快！
4）关于溢出（自己确保自己的调用所得到的结果不会溢出，一定是自己确保的，计算机不会给你做检查）
5）位运算玩法很多很多，特别是异或运算、如何用位运算实现加减乘除
```


## 负数的二进制表达

1、计算机中存储负数采用补码形式存储，两种方式推出负数x的二进制表示：
- 第一种：正数(-x)的二级制表示-1再全部取反。(或者记成取反加1)
- 第二种：第一位是1，表示负数，剩余位表示成$-2^n+(-x)$
- 第三种：1 << N + x, N表示位数

例如：
- -5 = 0101-1 -> 0100 取反-> 1011
- -5 = -2^3 + 3 -> 1011 (其中011表示3)
- -5 = 10000 (16) + (-5) = 11 = 1011

给一个负的二进制数怎么看是多少呢：取反加一（和上面一致）
- 1111 1011 -> 0000 0100 + 1 -> 0000 0101 -> -5

2、为什么用补码存储呢？
- 可以统一加减法运算
- 0只有一种表示方式
- 便于计算机硬件实现

例如：5+（-5）=0的计算过程
```text
   0000 0101  (5)
 + 1111 1011  (-5)
 = 0000 0000  (0)  （超出8位的进位被舍弃）
 ```

3、我们看看python的负数怎么表示呢？

In [3]:
print(bin(-5))

-0b101


你会发现python中二进制存储并没表示成最高位为1，而是直接显示负号。后面的二进制是正数对应的二进制

Python的整数没有固定的位数限制（可以任意大）

不同位数下-5的补码表示：
- 4位：  1011
- 8位：  1111 1011
- 16位： 1111 1111 1111 1011

4、特殊负数
```text
-1:    1111 1111  （所有位都是1）
-128:  1000 0000  （8位中最小的负数）
-127:  1000 0001
```

In [21]:
def print_binary(num, bits=32):
    """
    Print a number in its 32-bit binary representation.
    Left side is high bits, right side is low bits.
    """
    for i in range(bits-1, -1, -1):
        # We can use either:
        # print('1' if (num & (1 << i)) != 0 else '0', end='')
        # But not:
        # print('1' if (num & (1 << i)) == 1 else '0', end='')
        # Because if the i-th bit is 1, (num & (1 << i)) equals 2^i, not 1
        print('1' if (num & (1 << i)) != 0 else '0', end='')
    print()
    
a = 78
print(a)
print_binary(a)
print("===a===")

b = -6
print(b)
print_binary(b)
print("===b===")

# Binary literal
c = 0b1001110
print(c)
print_binary(c)
print("===c===")

78
00000000000000000000000001001110
===a===
-6
11111111111111111111111111111010
===b===
78
00000000000000000000000001001110
===c===


In [4]:
def print_twos_complement(n, bits=8):
    """
    按补码形式打印数字（支持正数和负数）
    参数:
        n: 要打印的数字
        bits: 位数（默认8位）
    """
    if n >= 0:
        # 正数直接转换为二进制，并补齐位数
        return format(n, f'0{bits}b')
    else:
        # 负数需要计算补码
        # (1 << bits) 给出指定位数的模，如8位时为256
        # 在模下加上负数，自动得到补码表示
        return format((1 << bits) + n, f'0{bits}b')
def test_binary_representation():
    numbers = [5, -5, 3, -3, 127, -128]
    bit_sizes = [4, 8, 16]
    
    for bits in bit_sizes:
        print(f"\n{bits}位二进制表示:")
        print("-" * 30)
        for n in numbers:
            try:
                binary = print_twos_complement(n, bits)
                print(f"{n:4d} 的补码表示: {binary}")
            except ValueError:
                print(f"{n:4d} 超出了{bits}位能表示的范围")

test_binary_representation()


4位二进制表示:
------------------------------
   5 的补码表示: 0101
  -5 的补码表示: 1011
   3 的补码表示: 0011
  -3 的补码表示: 1101
 127 的补码表示: 1111111
-128 的补码表示: -1110000

8位二进制表示:
------------------------------
   5 的补码表示: 00000101
  -5 的补码表示: 11111011
   3 的补码表示: 00000011
  -3 的补码表示: 11111101
 127 的补码表示: 01111111
-128 的补码表示: 10000000

16位二进制表示:
------------------------------
   5 的补码表示: 0000000000000101
  -5 的补码表示: 1111111111111011
   3 的补码表示: 0000000000000011
  -3 的补码表示: 1111111111111101
 127 的补码表示: 0000000001111111
-128 的补码表示: 1111111110000000


In [5]:
print_twos_complement(78)
print_twos_complement(~78)
print_twos_complement(-78) # 负数补码第三种获取方式
print_twos_complement(~78+1) # 负数补码第一种获取方式
print_twos_complement(~(78-1)) # 第一种

'01001110'

'10110001'

'10110010'

'10110010'

'10110010'

In [6]:
# 相反数：取反+1
# 注意的是：有符号数的最小值的相反数还是自己（对于固定位数来说的）
print_twos_complement(-128)
print_twos_complement(~(-128)+1) 
# 与-128的补码形式一样，如果对于固定位数来说，该补码形式仍然是-128
print_twos_complement(~(-128))

'10000000'

'10000000'

'01111111'

In [7]:
print(~(-128)+1) # 对python没有这个问题，因为python的整数没有位数限制

128


In [19]:
def demonstrate_hex():
    """演示Python中16进制数的各种操作"""
    
    print("1. 十进制转16进制:")
    num = 255
    # 方法1：使用hex()函数
    print(f"{num} 的16进制是: {hex(num)}")              # 0xff
    # 方法2：使用format
    print(f"{num} 的16进制是: {format(num, 'x')}")      # ff (小写)
    print(f"{num} 的16进制是: {format(num, 'X')}")      # FF (大写)
    # 方法3：使用f-string
    print(f"{num} 的16进制是: {num:x}")                 # ff (小写)
    print(f"{num} 的16进制是: {num:X}")                 # FF (大写)
    
    print("\n2. 指定位数的16进制(补零):")
    num = 26
    print(f"{num} 的16进制(4位): {format(num, '04x')}")  # 001a
    print(f"{num} 的16进制(4位): {num:04x}")            # 001a
    
    print("\n3. 带0x前缀的16进制:")
    print(f"{num} 的16进制: {format(num, '#x')}")       # 0x1a
    print(f"{num} 的16进制: {num:#x}")                  # 0x1a
    
    print("\n4. 16进制转十进制:")
    hex_num = "1a"
    print(f"16进制 {hex_num} 转十进制: {int(hex_num, 16)}")  # 26
    hex_num = "0x1a"
    print(f"16进制 {hex_num} 转十进制: {int(hex_num, 16)}")  # 26
    
    print("\n5. 负数的16进制表示:")
    num = -26
    print(f"{num} 的16进制是: {hex(num)}")              # -0x1a
    # 按补码形式显示（8位）
    print(f"{num} 的16进制补码(8位): {num & 0xFF:02x}")  # e6
    # 按补码形式显示（16位）
    print(f"{num} 的16进制补码(16位): {num & 0xFFFF:04x}")  # ffe6
    
    print("\n6. 16进制字节数据:")
    data = bytes([26, 255, 0, 127])
    print(f"字节数据的16进制: {data.hex()}")            # 1aff007f
    print(f"字节数据的16进制(带空格): {data.hex(' ')}")  # 1a ff 00 7f
    
    print("\n7. 16进制字符串转字节:")
    hex_string = "1aff007f"
    print(f"16进制字符串转字节: {bytes.fromhex(hex_string)}")
    
    print("\n8. 16进制数值运算:")
    a = 0x1a  # 直接使用16进制字面量
    b = 0xff
    print(f"0x1a + 0xff = {hex(a + b)}")  # 0x119


demonstrate_hex()

1. 十进制转16进制:
255 的16进制是: 0xff
255 的16进制是: ff
255 的16进制是: FF
255 的16进制是: ff
255 的16进制是: FF

2. 指定位数的16进制(补零):
26 的16进制(4位): 001a
26 的16进制(4位): 001a

3. 带0x前缀的16进制:
26 的16进制: 0x1a
26 的16进制: 0x1a

4. 16进制转十进制:
16进制 1a 转十进制: 26
16进制 0x1a 转十进制: 26

5. 负数的16进制表示:
-26 的16进制是: -0x1a
-26 的16进制补码(8位): e6
-26 的16进制补码(16位): ffe6

6. 16进制字节数据:
字节数据的16进制: 1aff007f
字节数据的16进制(带空格): 1a ff 00 7f

7. 16进制字符串转字节:
16进制字符串转字节: b'\x1a\xff\x00\x7f'

8. 16进制数值运算:
0x1a + 0xff = 0x119


## 常用位运算

In [9]:

# 1. 按位与 &
print("1. 按位与 &:")
a = 60      # 二进制: 0011 1100
b = 13      # 二进制: 0000 1101
print(f"60 & 13 = {a & b}")  # 结果: 12 (0000 1100)
print(f"二进制表示: {bin(a)} & {bin(b)} = {bin(a & b)}\n")


1. 按位与 &:
60 & 13 = 12
二进制表示: 0b111100 & 0b1101 = 0b1100



In [10]:

# 2. 按位或 |
print("2. 按位或 |:")
print(f"60 | 13 = {a | b}")  # 结果: 61 (0011 1101)
print(f"二进制表示: {bin(a)} | {bin(b)} = {bin(a | b)}\n")


2. 按位或 |:
60 | 13 = 61
二进制表示: 0b111100 | 0b1101 = 0b111101



In [11]:

# 3. 按位异或 ^
print("3. 按位异或 ^:")
print(f"60 ^ 13 = {a ^ b}")  # 结果: 49 (0011 0001)
print(f"二进制表示: {bin(a)} ^ {bin(b)} = {bin(a ^ b)}\n")


3. 按位异或 ^:
60 ^ 13 = 49
二进制表示: 0b111100 ^ 0b1101 = 0b110001



In [12]:

# 4. 按位取反 ~
print("4. 按位取反 ~:")
print(f"~60 = {~a}")  # 结果: -61
print(f"二进制表示: ~{bin(a)} = {bin(~a & 0xFFFFFFFF)}\n")  # 显示32位表示


4. 按位取反 ~:
~60 = -61
二进制表示: ~0b111100 = 0b11111111111111111111111111000011



In [13]:

# 5. 左移 << 左移n位相当于乘以2^n
print("5. 左移 <<:")
x = 5       # 二进制: 0000 0101
print(f"5 << 2 = {x << 2}")  # 结果: 20 (0001 0100)
print(f"二进制表示: {bin(x)} << 2 = {bin(x << 2)}\n")


5. 左移 <<:
5 << 2 = 20
二进制表示: 0b101 << 2 = 0b10100



In [17]:

# 6. 右移 >> 算数右移（用符号位来补）
print("6. 右移 >>:")
y = 20      # 二进制: 0001 0100
print(f"20 >> 2 = {y >> 2}")  # 结果: 5 (0000 0101)
print(f"二进制表示: {bin(y)} >> 2 = {bin(y >> 2)}\n")

y = -5     
print(f"-5 >>  = {y >> 1}")  
print(f"二进制表示: {bin(y)} >> 1 = {bin(y >> 2)}\n")


6. 右移 >>:
20 >> 2 = 5
二进制表示: 0b10100 >> 2 = 0b101

-5 >>  = -3
二进制表示: -0b101 >> 1 = -0b10



## 常用属性

### a^a=0 

In [22]:
# 交换两个数(使用的属性a^a=0,需要注意a和b不能指向同一地址或同一个数)
a = 5
b = 10
print(f"交换前: a = {a}, b = {b}")
a = a ^ b
b = a ^ b
a = a ^ b
print(f"交换后: a = {a}, b = {b}")

交换前: a = 5, b = 10
交换后: a = 10, b = 5


## 技巧

In [ ]:

# 实际应用示例
print("实际应用示例:")

# 1. 判断奇偶性
num = 7
print(f"{num} 是{'偶数' if (num & 1) == 0 else '奇数'}")



# 3. 设置标志位
flags = 0
READ = 4    # 二进制: 0100
WRITE = 2   # 二进制: 0010
EXEC = 1    # 二进制: 0001

# 设置权限
flags |= READ | WRITE     # 添加读写权限
print(f"\n权限设置示例:")
print(f"当前权限: {bin(flags)}")
print(f"是否有读权限: {bool(flags & READ)}")
print(f"是否有写权限: {bool(flags & WRITE)}")
print(f"是否有执行权限: {bool(flags & EXEC)}")
